# Bibliotecas

In [1]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense

2024-03-11 22:41:29.593957: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 22:41:29.637532: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 22:41:29.637585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 22:41:29.639057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 22:41:29.645950: I tensorflow/core/platform/cpu_feature_guar

# Abrindo dados

In [2]:
x = np.float64(np.load(open('data/x.npy', 'rb')))
y = np.int8(np.load(open('data/y.npy', 'rb')))

In [3]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

# Criar Modelo

In [7]:
modelo = Sequential()

modelo.add(LSTM(32, input_shape=(3006, 1)))
modelo.add(Dense(256, activation="relu"))
modelo.add(Dense(3, activation="softmax"))

modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

modelo.summary()

2024-03-11 22:42:54.704847: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                4352      
                                                                 
 dense_6 (Dense)             (None, 256)               8448      
                                                                 
 dense_7 (Dense)             (None, 3)                 771       
                                                                 
Total params: 13571 (53.01 KB)
Trainable params: 13571 (53.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Treinar Modelo

In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/lstm/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=20, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('models/LSTMmodel.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=200,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)

Epoch 1/200


2024-03-11 22:43:02.084962: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-11 22:43:02.446610: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6bf01717f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 22:43:02.446667: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-11 22:43:02.474107: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710211382.603681  439798 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


234/234 [==============================] - 56s 230ms/step - loss: 1.0807 - accuracy: 0.3900 - val_loss: 1.0781 - val_accuracy: 0.4002
Epoch 2/200


/home/aurelio/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


234/234 [==============================] - 48s 206ms/step - loss: 1.0762 - accuracy: 0.3991 - val_loss: 1.0762 - val_accuracy: 0.3990
Epoch 3/200
234/234 [==============================] - 51s 220ms/step - loss: 1.0758 - accuracy: 0.3995 - val_loss: 1.0769 - val_accuracy: 0.4003
Epoch 4/200
234/234 [==============================] - 50s 215ms/step - loss: 1.0749 - accuracy: 0.4022 - val_loss: 1.0751 - val_accuracy: 0.3973
Epoch 5/200
234/234 [==============================] - 50s 215ms/step - loss: 1.0743 - accuracy: 0.4010 - val_loss: 1.0755 - val_accuracy: 0.4043
Epoch 6/200
234/234 [==============================] - 50s 212ms/step - loss: 1.0737 - accuracy: 0.4055 - val_loss: 1.0754 - val_accuracy: 0.4049
Epoch 7/200
234/234 [==============================] - 49s 208ms/step - loss: 1.0725 - accuracy: 0.4077 - val_loss: 1.0731 - val_accuracy: 0.4084
Epoch 8/200
234/234 [==============================] - 52s 222ms/step - loss: 1.0714 - accuracy: 0.4065 - val_loss: 1.0716 - val_accurac

In [ ]:
# modelo.save('models/tf-cnn-model')